# Data preprocessing and model preparation

In [1]:
library(data.table) #fread
library(dplyr)
library(geosphere) # Calculate vehicle distance
library(lubridate)# Date column extraction

Warning message:
"package 'data.table' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'geosphere' was built under R version 3.6.3"

Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following object is masked from 'package:base':

    date




In [2]:
# adjust the R limit
memory.limit(size=249200)

[1] 249200

### Input location data and data preprocessing

In [4]:
# Read data from folder
# Suggestion: Just do one year at one time. One year's table has 94363502 rows(green)
YEAR = c('19')
MONTH= c("01") #FOR FULL TABLE
DISTANCE_FILEPATH = "../../data/raw/vehicle-location/"

In [3]:
# aggregrate_trajectory_table
aggregate_line_trajectories = function(year, month){
    assign("dg", fread(paste(DISTANCE_FILEPATH, paste("lightrail", "trajectories", month, year, ".csv", sep = "-", collapse = ""), sep="")))
    assign("dh", fread(paste(DISTANCE_FILEPATH, paste("heavyrail", "trajectories", month, year, ".csv", sep = "-", collapse = ""), sep="")))
    # Combine the original tables to a single one for analysis
    dg = subset(dg, select = c(trxtime, trainid, lineid, lat, lon))
    dh = subset(dh, select = c(trxtime, trainid, lineid, lat, lon))
    df = rbind(dg, dh)
    return(df)
}

In [5]:
trajectroy_table = aggregate_line_trajectories(YEAR,MONTH)

In [6]:
dim(trajectroy_table)

[1] 9116148       5

### Experimental section

In [7]:
# Data preprocessing
preprocess_data = function(df){
    # Add time column to do analysis by random time scale
    df$day = day(df$trxtime)
    df$month = month(df$trxtime)
    df$year = year(df$trxtime)
    # check the na values ratio
    df_na = df[(is.na(df$lon)) | (is.na(df$lat)),]
    df_na_rate = nrow(df_na)/nrow(df)
    print(df_na_rate)
    # remove the rows with values of lat and lon are 0
    df = df[!(is.na(df$lon)) | !(is.na(df$lat)),] 
    df = df[order(df$trainid, df$trxtime),]
    df = distinct(df, trxtime, trainid, .keep_all = TRUE) # Remove the duplicated time record
    return(df)
}

In [9]:
trajectroy_table = preprocess_data(trajectroy_table)

[1] 0


In [10]:
dim(trajectroy_table)

[1] 6617969       8

In [13]:
d_test = trajectroy_table[1:2000,]

In [14]:
dim(d_test)

[1] 2000    8

In [ ]:
# All speed are mph
# Green line
## Branch main 8.6
## Branch B 7.1
## Branch C 7.9
## Branch D 18.7
## Branch E 7.1
# Red line 
## A 16.7
## B 20.1
# Orange line 18.3
# Blue line 18.6
# Ref: https://www.fixmbta.com/green-line-upgrades

### Vehicle distance,speed, acceleration calculation

In [58]:
# Function to compute distances (D), speeds (S) and acceleration (A) in meters, meters per second, km per hour and m s^-2
compute_trajectories <- function(d) {
    d$dist_meters = NA
    d$dist_meters = as.numeric(d$dist_meters)
    d$interval_seconds = NA
    d$d$interval_seconds = as.numeric(d$d$interval_seconds)
    d$speed_mps = NA
    d$speed_mps = as.numeric(d$speed_mps)
    d$speed_kph = NA
    d$speed_kph = as.numeric(d$speed_kph)
    d$accel_mps2 = NA
    d$accel_mps2 = as.numeric(d$accel_mps2)
    n <- nrow(d)
    if (n >= 2) {
        # Compute interval distance using Haversine function
        d$dist_meters[2:n] = distHaversine(cbind(d$lon[1:n-1],d$lat[1:n-1]),cbind(d$lon[2:n],d$lat[2:n]))
        # Compute time interval
        d$interval_seconds[2:n] = as.numeric(difftime(d$trxtime[2:n], d$trxtime[1:n-1], units = "secs"))
        # Compute speed in meters per second
        d$speed_mps[2:n] = d$dist_meters[2:n] / d$interval_seconds[2:n]
        # Convert speed to kph
        d$speed_kph[2:n] = d$speed_mps[2:n]*3.6
        # Compute accelerations
        d$accel_mps2[2:n] = (d$speed_mps[2:n] - d$speed_mps[1:n-1])/d$interval_seconds[2:n]
        # Spurious values statistics
        diagnostics = {}
        index_excessive_speeds = as.numeric(row.names(d[d$speed_kph > 128 & !is.na(d$speed_kph), ]))
        diagnostics$mean_excess_speed = mean(as.numeric(d[index_excessive_speeds, "speed_kph"]))
        diagnostics$min_excess_speed = min(as.numeric(d[index_excessive_speeds, "speed_kph"])) 
        diagnostics$max_excess_speed = max(as.numeric(d[index_excessive_speeds, "speed_kph"])) 
        diagnostics$num_excess_speed = length(d[index_excessive_speeds, "speed_kph"]) 
        diagnostics$prop_excess_speed = round(100*diagnostics$num_excess_speed/n, 2)
        # you should add other metrics for original and corrected 
        print(diagnostics)
         # correct the wrong values
        d[index_excessive_speeds, "speed_mps"] = d[index_excessive_speeds - 1, "speed_mps"]
        d[index_excessive_speeds, "dist_meters"] = d[index_excessive_speeds, "speed_mps"] * d[index_excessive_speeds,"interval_seconds"]
        d[index_excessive_speeds, "accel_mps2"] = (d[index_excessive_speeds, "speed_mps"] 
                                                   - d[index_excessive_speeds - 1, "speed_mps"]) / d[index_excessive_speeds,"interval_seconds"]
     }
    return(d)
}
# summary_computation = data.frame(outlier_rate = nrow(d_outlier)/nrow(d),outlier_mean = mean(d_outlier$speed_kph),outlier_max = max(d_outlier$speed_kph),outlier_min = min(d_outlier$speed_kph),max_correct = max(d$speed_kph),
#                          min_correct = min(d$speed_kph),mean_correct = (d$speed_kph))

In [59]:
# Process calculation for four lines
process_month_trajectory = function(data){
    results_df = data.frame() # empty dataframe
    diagnostics_df = data.frame()
    for(i in unique(data$day)) { 
        #if (i <= 31) { # define when the loop will end
        data_day <- data[data$day == i, ]
        for (j in unique(data_day$trainid)) {# Put each train in one loop in a subset
            data_day_train = data_day[data_day$trainid == j, ]        
            trajectory_and_diagnostics <- compute_trajectories(data_day_train) # Process data with only one row separately   
            trajectories = trajectory_and_diagnostics
            #diagnostics = trajectory_and_diagnostics[2]            
            results_df <- rbind(results_df, trajectories) 
            # diagnostics_df <- rbind(diagnostics_df, diagnostics) 
            #}
        }
    }
   # write.csv(x=results_df, file.path("../../data/tidy/", paste("trajectories", yy, mm,".csv",sep = "-",collapse = "")))
   # write.csv(x=diagnostics_df, file.path("../../data/tidy/", paste("trajectory-diagnostics",yy,mm,".csv",sep = "-",collapse = "")))
    # return(list(results_df,diagnostics_df))
    return(results_df)
 }

In [60]:
process_month_trajectory(d_test)

$mean_excess_speed
[1] NA

$min_excess_speed
[1] NA

$max_excess_speed
[1] NA

$num_excess_speed
[1] 1

$prop_excess_speed
[1] 0.07

$mean_excess_speed
[1] NA

$min_excess_speed
[1] NA

$max_excess_speed
[1] NA

$num_excess_speed
[1] 1

$prop_excess_speed
[1] 0.81

$mean_excess_speed
[1] NA

$min_excess_speed
[1] NA

$max_excess_speed
[1] NA

$num_excess_speed
[1] 1

$prop_excess_speed
[1] 0.2



trxtime,trainid,lineid,lat,lon,day,month,year,dist_meters,interval_seconds,d,speed_mps,speed_kph,accel_mps2
<chr>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<list>,<dbl>,<dbl>,<dbl>
2019-01-01 04:00:00,10001,4,42.33701,-71.25349,1,1,2019,NA,NA,,NA,NA,NA
2019-01-01 04:42:35,10001,4,42.33613,-71.25391,1,1,2019,104.64023,2555,,0.040955079,0.14743828,NA
2019-01-01 04:45:05,10001,4,42.33603,-71.25436,1,1,2019,38.35157,150,,0.255677118,0.92043762,1.431480e-03
2019-01-01 05:49:31,10001,4,42.33613,-71.25391,1,1,2019,38.35157,3866,,0.009920219,0.03571279,-6.356878e-05
2019-01-01 05:49:42,10001,4,42.33635,-71.25379,1,1,2019,26.23150,11,,2.384681862,8.58485470,2.158874e-01
2019-01-01 06:19:33,10001,4,42.33658,-71.25367,1,1,2019,28.08000,1791,,0.015678392,0.05644221,-1.322727e-03
2019-01-01 06:35:24,10001,4,42.33680,-71.25352,1,1,2019,26.95307,951,,0.028341823,0.10203056,1.331591e-05
2019-01-01 06:35:36,10001,4,42.33696,-71.25329,1,1,2019,26.68781,12,,2.223984489,8.00634416,1.829702e-01
2019-01-01 06:39:05,10001,4,42.33633,-71.25019,1,1,2019,264.84088,209,,1.267181256,4.56185252,-4.578006e-03


In [29]:
d_test

trxtime,trainid,lineid,lat,lon,day,month,year
<chr>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
2019-01-01 04:00:00,10001,4,42.33701,-71.25349,1,1,2019
2019-01-01 04:42:35,10001,4,42.33613,-71.25391,1,1,2019
2019-01-01 04:45:05,10001,4,42.33603,-71.25436,1,1,2019
2019-01-01 05:49:31,10001,4,42.33613,-71.25391,1,1,2019
2019-01-01 05:49:42,10001,4,42.33635,-71.25379,1,1,2019
2019-01-01 06:19:33,10001,4,42.33658,-71.25367,1,1,2019
2019-01-01 06:35:24,10001,4,42.33680,-71.25352,1,1,2019
2019-01-01 06:35:36,10001,4,42.33696,-71.25329,1,1,2019
2019-01-01 06:39:05,10001,4,42.33633,-71.25019,1,1,2019


In [ ]:
main = function() {
    for (y in YEARLIST) {
        for (m in MONTHLIST) {
            df_agg = aggregate_line_trajectories(y, m)
            df_agg = preprocess(df_agg)
            process_month_trajectory(df_agg, y, m)            
        }
    }
}


### Experimental section

In [ ]:
#  # Count the row numbers which lat and lon are NA
#      df_na = df[(is.na(data$lon)) | (is.na(data$lat)),]
#      number_na = nrow(df_na)

In [ ]:
# # Process value correction
#         for(k in 2:n) { 
#                 # revise the incorrect values 
#                 if (!(is.na(d$speed_kph[k]))) {
#                     if (d$speed_kph[k] > 128 ){
#                        # the reference link is here:
#                        # https://www.masslive.com/worcester/2019/09/electrified-faster-and-more-trains-7-ways-the-mbtas-commuter-rail-service-could-be-a-lot-better.html
#                          if(k==3){d$speed_kph[k] = d$speed_kph[k+1]}
#                          else{
#                         d$speed_kph[k] = (d$speed_kph[k-1] + d$speed_kph[k+1] )/2
#                         d$speed_mps[k] = d$speed_kph[k] / 3.6
#                         d$accel_mps2[k] = (d$speed_mps[k]-d$speed_mps[k-1]) / d$interval_seconds[k]
#                         d$dist_meters[k] = d$speed_mps[k] * d$interval_seconds[k]}
#                         else if (k == nrows(d)){d$speed_kph[k] = d$speed_kph[k-1]}
#                     }
#                 }
#     }        